# 6. 학습 관련 기술들
## 6.1 매개변수 갱신
### 확률적 경사 하강법(SGD)<br> 
$$\mathbf{W} \leftarrow \mathbf{W}-\eta \frac{\partial L}{\partial \mathbf{W}}$$
$\mathbf{W}$ : 갱신할 가중치 매개변수<br>
$\frac{\partial L}{\partial \mathbf{W}}$ : W에 대한 손실함수의 기울기<br>
$\eta$ : 학습률 (0.01이나 0.001과 같은 값을 미리 정해서 사용)

In [ ]:
class SGD :
    def __init__(self, lr=0.01):
        self.lr=lr
        
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr*grads[key]

### ① 모멘텀
$$\mathbf{v} \leftarrow \alpha \mathbf{v}-\eta \frac{\partial L}{\partial \mathbf{W}}$$
$$\mathbf{W} \leftarrow \mathbf{W}+\mathbf{v}$$
$\mathbf{v}$ : 속도<br>
$\alpha$ : 0.9 등의 값으로 설정

In [ ]:
class Momentum :
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr=lr
        self.momentum=momentum
        self.v=None
        
    def update(self, params, grads):
        if self.v is None:
            self.v={}
            for key, val in params.items():
                self.v[key]=np.zeros_like(val)
                
        for key in params.keys():
                self.v[key]=self.momentum*self.v[key]-self.lr*grads[key]
                params[key]+=self.v[key]

### ② AdaGrad
$$\mathbf{h}\leftarrow \mathbf{h}+\frac{\partial L}{\partial \mathbf{W}}\odot \frac{\partial L}{\partial \mathbf{W}}$$
$$\mathbf{W} \leftarrow \mathbf{W}-\eta \frac{1}{\sqrt{\mathbf{h}}} \frac{\partial L}{\partial \mathbf{W}}$$
$\odot$ : 행렬의 원소별 곱셈<br>
$\mathbf{h}$ : 기존 기울기 값을 제곱해 계속 더해줌 (크게 갱신된 원소는 학습률이 낮아짐)

In [ ]:
class AdaGrad:
    def __init__(self, lr=0.01):
        self.lr=lr
        self.h=None
        
    def update(self, params, grads):
        if self.h is None:
            self.h={}
            for key, val in params.items():
                self.h[key]=np.zeros_like(val)
                
        for key in params.keys():
            self.h[key]+=grads[key]*grads[key]
            params[key]-=self.lr*grads[key]/(np.sqrt(self.h[key])+1e-7)

### ③ Adam = 모멘텀 + AdaGrad
<br><br><br>
$$f(x,y)=\frac{1}{20}x^{2}+y^{2}$$
![그림6-8](https://raw.githubusercontent.com/asd93/a/master/p/fig%206-8.png)

## 6.2 가중치의 초깃값
초깃값을 모두 0으로 하면 오차역전파법에서 모든 가중치의 값이 똑같이 갱신, 갱신을 거쳐도 같은 값 유지<br>
무작위로 설정해야 함

In [ ]:
# 가중치의 초깃값에 따라 은닉층 활성화값들이 어떻게 변하는지
# 활성화함수로 시그모이드 함수 사용하는 5층 신경망

import numpy as np
import matplotlib.pyplot as plt

def sigmoid(x):
    return 1/(1+np.exp(-x))

x=np.random.randn(1000,100) # 1000개 데이터
node_num=100 # 각 은닉층 노드 수 
hidden_layer_size=5 # 은닉층 수
activations={} # 활성화값 저장

for i in range(hidden_layer_size):
    if i != 0:
        x=activations[i-1]
        
    w=np.random.randn(node_num, node_num)*1    # 가중치 표준편차 1
    a=np.dot(x,w)
    z=sigmoid(a)
    activations[i]=z
    
# 히스토그램 그리기

for i, a in activations.items():
    plt.subplot(1,len(activations), i+1)
    plt.title(str(i+1)+"-layer")
    plt.hist(a.flatten(), 30, range(0,1))
plt.show()

시그모이드 함수는 출력값이 0이나 1에 가까워지면 미분이 0으로 다가감.<br>
데이터가 0과 1에 치우쳐 분포하게 되면 역전파 기울기 값이 작아지다가 사라짐 - **기울기 소실**

In [ ]:
x=np.random.randn(1000,100) # 1000개 데이터
node_num=100 # 각 은닉층 노드 수 
hidden_layer_size=5 # 은닉층 수
activations={} # 활성화값 저장

for i in range(hidden_layer_size):
    if i != 0:
        x=activations[i-1]
        
    w=np.random.randn(node_num, node_num)/np.sqrt(node_num)    # Xavior 초깃값 - 표준편차가1/n^0.5
    a=np.dot(x,w)
    z=sigmoid(a)
    activations[i]=z
    
    
for i, a in activations.items():
    plt.subplot(1,len(activations), i+1)
    plt.title(str(i+1)+"-layer")
    plt.hist(a.flatten(), 30, range(0,1))
plt.show()

# 넓게 분포

일그러짐은 sigmoid 대신 tanh 이용하면 개선

### Sigmoid를 사용할 때의 가중치 초깃값
**Xavior 초깃값**<br>
앞 계층 노드가 n개일 때, 표준편차가 $\frac{1}{\sqrt{n}}$ 인 정규분포 사용

### ReLU를 사용할 때의 가중치 초깃값
**He 초깃값**<br>
앞 계층 노드가 n개일 때, 표준편차가 $\sqrt{\frac{2}{n}}$ 인 정규분포 사용

## 6.3 배치 정규화
각 층에서의 활성화 값이 적당히 분포되도록 조정
* 학습 속도 개선
* 초깃값에 크게 의존하지 않음
* 오버피팅 억제

미니배치 단위로 정규화<br>
데이터 분포 평균 0, 분산 1 되도록 정규화<br>
$$\mu _{B}\leftarrow \frac{1}{m}\sum_{i=1}^{m}x_{i}$$
$$\sigma _{B} ^{2}\leftarrow \frac{1}{m}\sum_{i=1}^{m}(x_{i}-\mu_{B})^2$$
$$\hat{x_i}\leftarrow \frac{x_i-\mu_B}{\sqrt{\sigma_B^2+\varepsilon }}$$

## 6.4 바른 학습을 위해
### 오버피팅
* 매개변수가 많고 표현력이 높은 모델
* 훈련 데이터가 적음

In [ ]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True)

x_train=x_train[:300]
t_train=t_train[:300]

In [ ]:
network=MultiLayerNet(input_size=784, hidden_size_list[100,100,100,100,100,100], output_size=10)
optimizer=SGD(lr=0.01)
max_epochs=201
train_size=x_train.shape[0]
batch_size=100

train_loss_list=[]
train_acc_list=[]
test_acc_list=[]

iter_per_epoch=max(train_size/batch_size, 1)
epoch_cnt=0

for i in range(1000000000):
    batch_mask=np.random.choice(train_size,batch,size)
    x_batch=x_train[batch_mask]
    t_batch=t_train[batch_mask]
    
    grads=network.gradient(x_batch, t_batch)
    optimizer.update(network.params, grads)
    
    if i % iter_per_epoch ==0:
        train_acc=network.accuracy(x_train, t_train)
        test_acc=network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        
        epoch_cnt += 1
        if epoch_cnt >= max_epochs:
            break
            
# train_acc_list 와 test_acc_list 에 에폭 단위의 정확도 저장

![그림 6-20](https://raw.githubusercontent.com/asd93/a/master/p/fig%206-20.png)

### 가중치 감소
큰 가중치에 대해서 그에 상응하는 큰 페널티를 부과하여 오버피팅을 억제<br><br>
모든 가중치 각각의 손실함수에 $\frac{1}{2}\lambda \mathbf{W}^2$ 를 더함<BR>
$\lambda$ : 가중치 세기 조절하는 하이퍼파라미터

### 드롭아웃
훈련 때 은닉층의 뉴런을 무작위로 골라 삭제<br>
시험 때는 모든 뉴런에 신호를 전달하고 각 뉴런의 출력에 훈련 때 삭제 안 한 비율을 곱하여 출력<br>

In [ ]:
class Dropout:
    def __init__(self,dropout_ratio=0.5):
        self.dropout_ratio+dropout_ratio
        self.mask=None
        
    def forward(self, x, train_fig=True):
        if train_fig:
            self.mask=np.random.rand(*x.shape) > self.dropout_ratio
            return x*self.mask
        else:
            return x+(1.0-self.dropout_ratio)
        
    def backward(self, dout):
        return dout*self.mask

## 6.5 적절한 하이퍼파라미터 값 찾기
### 검증 데이터
* 훈련 데이터:매개변수 학습
* 검증 데이터:하이퍼파라미터 성능 평가
* 시험 데이터:신경망의 범용 성능 평가

In [ ]:
(x_train, t_train),(x_test,t_test)=load_mnist()

# 훈련 데이터를 뒤섞는다.
x_train, t_train= shuffle_dataset(x_train, t_train)

# 20%를 검증 데이터로 분할
validation_rate=0.20
validation_num=int(x_train.shape[0]*validation_rate)

x_val=x_train[:validation_num]
t_val=t_train[:validation_num]
x_train=x_train[validation_num:]
t_train=t_train[validation_num:]